<a href="https://colab.research.google.com/github/oswaldochan/COVID19-Tweet-Toxic-Classification/blob/main/02_NLP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install xgboost

In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# los REALES 😎
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import plotly.express as px

import sklearn as sk
import xgboost as xgb


## Reading data

In [2]:
data = pd.read_csv('train.csv.zip')

In [3]:
def add_y_col(dataframe):
    d_frame = dataframe.copy()
    # columns that are more or less toxic
    cols_info = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    # If the row has one or more negative categories, it is considered as Y = negative 
    d_frame['Y'] = d_frame[cols_info].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    num_neg = d_frame['Y'].value_counts()[1]
    positive = d_frame[d_frame['Y'] == 0].sample(num_neg)[['comment_text', 'Y']].reset_index(drop=True)
    negative = d_frame[d_frame['Y'] == 1][['comment_text', 'Y']].sample(frac=1).reset_index(drop=True)

    # intercalate rows
    return pd.concat([positive, negative]).sort_index(kind='merge').reset_index(drop=True)


In [4]:
print(f'Total number of rows in the training dataset: {data.shape[0]}')

Total number of rows in the training dataset: 159571


## Clean and transform data

In [46]:
df = add_y_col(data)
X_raw = df['comment_text']
y = df['Y'].to_numpy().reshape((-1, 1)).astype(float)

In [47]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, 
                                                    y,
                                                    test_size = 0.20)

In [7]:
# Taken from documentation 
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
MAX_COLS = 2000
# Create a Tokenizer object
## This tokenizer will learn the 'num_words' most popular words in the vocab and
## assign a tfidf metric to it
token = Tokenizer(num_words=MAX_COLS,
                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                  lower = True, split = ' ', oov_token = "XDDD",
                  document_count = 0
                  )

token.fit_on_texts(X_train_raw)

X_train_matrix = token.texts_to_matrix(X_train_raw, mode='tfidf')
X_test_matrix = token.texts_to_matrix(X_test_raw, mode='tfidf')

In [67]:
# Taken from documentation 
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
MAX_WORDS = 3000
MAX_LEN = 120

# Create a Tokenizer object
## This tokenizer will learn the most MAX_WORDS words and
## assign an integer in order to those words
token_seq = Tokenizer(
                  num_words=MAX_WORDS,
                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                  lower = True, split = ' ', oov_token = "XDDD",
                  )

token_seq.fit_on_texts(X_train_raw)

X_train_seq = token_seq.texts_to_sequences(X_train_raw)
X_test_seq = token_seq.texts_to_sequences(X_test_raw)

X_train_final = pad_sequences(X_train_seq, padding = 'post', maxlen = MAX_LEN)
X_test_final = pad_sequences(X_test_seq, padding = 'post', maxlen = MAX_LEN)

In [75]:
import io 
import json

## TO SAVE THE VOCABULARY LEARNED BY THE TOKENIZER
tokenizer_json = token_seq.to_json()
with io.open('/content/config_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [222]:
# vocab = {} 
# for i in X_train_seq:
#     for n in i:
#         if n not in vocab.keys():
#             vocab[n] = 1
#         else:
#             vocab[n] += 1

In [217]:
# (np.array([len(i) for i in X_train_seq]).astype(int) > 300).sum()

734

## Training the model

In [25]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.74.211.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.211.66:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [26]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [70]:
 with strategy.scope():
    model = keras.models.Sequential()
    model.add(keras.layers.Embedding(MAX_WORDS, 4, input_length=X_train_final.shape[1]))
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(5, return_sequences=True, recurrent_dropout=0.5, dropout=0.5)))
    model.add(keras.layers.GlobalMaxPooling1D())
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

epochs = 15
# batch_size = int(X_train_final.shape[0] / 9)
batch_size = 512

history = model.fit(X_train_final, 
                    y_train, 
                    validation_data=(X_test_final, y_test),
                    epochs=epochs, batch_size=batch_size)

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_32 (Embedding)    (None, 120, 4)            12000     
                                                                 
 bidirectional_32 (Bidirecti  (None, 120, 10)          400       
 onal)                                                           
                                                                 
 global_max_pooling1d_32 (Gl  (None, 10)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_60 (Dense)            (None, 8)                 88        
                                                                 
 dense_61 (Dense)            (None, 1)                 9         
                                                                 
Total params: 12,497
Trainable params: 12,497
Non-tra

In [72]:
model.save('/content/modelo_entrenado_file.h5')

In [81]:
predsss = model.predict(X_test_final)

In [89]:
np.apply_along_axis(lambda x: 1 if x > 0.5 else 0, 1, predsss)

array([1, 0, 0, ..., 0, 0, 1])

In [90]:
predsss

array([[0.9960264 ],
       [0.1904687 ],
       [0.12465322],
       ...,
       [0.04369462],
       [0.04073295],
       [0.98269147]], dtype=float32)

## Trying with non-neural networks

In [20]:
model2 = xgb.XGBClassifier(objetive='binary:logistic', seed=1234,verbosity=3, n_jobs=-1)

In [21]:
model2.fit(X_train_matrix, np.ravel(y_train))

[21:24:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[21:24:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[21:24:38] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:24:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[21:24:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:24:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[21:24:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:24:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nod

XGBClassifier(n_jobs=-1, objetive='binary:logistic', seed=1234, verbosity=3)

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
rf = RandomForestClassifier(n_jobs=-1, verbose=3)

In [17]:
rf.fit(X_train_matrix, np.ravel(y_train))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    7.9s


building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.4s finished


RandomForestClassifier(n_jobs=-1, verbose=3)

In [14]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Reporting most important scores
def report_scores(y_true, y_pred):

    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)

    print(
        f'''\n\nThe weighted average precision is {precision:.{3}}\n
The weighted average recall is {recall:.{3}}\n
The weighted average f1-score is {f1:.{3}}\n
The accuracy is {accuracy:.{3}}
        '''

    )

In [19]:
report_scores(np.ravel(y_test), rf.predict(X_test_matrix))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.1s




The weighted average precision is 0.861

The weighted average recall is 0.859

The weighted average f1-score is 0.858

The accuracy is 0.859
        


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.3s finished


In [22]:
report_scores(np.ravel(y_test), model2.predict(X_test_matrix))



The weighted average precision is 0.844

The weighted average recall is 0.836

The weighted average f1-score is 0.835

The accuracy is 0.836
        
